In [2]:
#####Poisson Tucker Decomposition####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns

#np.random.seed(98537)

In [3]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [11]:
####データの発生####
##データの設定
k1 = 10
k2 = 10
k3 = 5
hh = 5000   #ユーザー数
item = 3000   #アイテム数
context = 50   #コンテキスト数
pt = np.random.poisson(np.random.gamma(30.0, 1/0.2, hh), hh)   #ユーザーあたりのレコード数
hhpt = np.sum(pt)
vec_k1 = np.repeat(1, k1)
vec_k2 = np.repeat(1, k2)
vec_k3 = np.repeat(1, k3)

In [13]:
hhpt

748929